# [M_01] Protokół: Multimodalny Analityk (Gemini 2.5 Flash)

Ten moduł implementuje logikę agenta analitycznego odpowiedzialnego za multimodalną analizę wideo. Wykorzystujemy model Gemini 2.5 Flash do bezpośredniej ekstrakcji metadanych bez transkrypcji pośredniej.

**Standard Techniczny:**
* **Framework:** PydanticAI (Standard `output_type`)
* **Silnik:** `GoogleModel` (gemini-2.5-flash)
* **Zmienna:** `GOOGLE_API_KEY` (Standard dla GoogleModel)

In [ ]:
import os
import sys
import time
import pydantic_ai
import google.generativeai as genai
from typing import List
from pydantic import BaseModel, Field
from pydantic_ai import Agent
from pydantic_ai.models.google import GoogleModel
from dotenv import load_dotenv

# 0. Weryfikacja wersji i ścieżek
print(f"🔍 Wersja pydantic-ai: {pydantic_ai.__version__}")

if os.getcwd().endswith("notebooks"):
    os.chdir("..")
sys.path.append(os.path.join(os.getcwd(), "src"))

# 1. Setup środowiska (plik .env w root projektu)
# Upewnij się, że w .env masz klucz GOOGLE_API_KEY
load_dotenv(".env")
os.environ["GOOGLE_API_KEY"] = os.getenv("GEMINI_API_KEY") # Mapowanie jeśli używasz innej nazwy

# 2. Inicjalizacja silnika (PydanticAI sam pobierze GOOGLE_API_KEY)
model = GoogleModel('gemini-2.5-flash')

# Konfiguracja SDK Google (do uploadu plików)
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

print(f"STATUS: System gotowy. Katalog roboczy: {os.getcwd()}")

## 1. Definicja Kontraktu Danych (Schema)

Tworzymy rygorystyczny model danych dla Agenta. Każdy wynik musi być instancją klasy `VideoAnalysisReport`.

In [2]:
# Uproszczone modele bez ograniczeń ge/le, które blokują API Google
class VideoClip(BaseModel):
    """Fragment wyselekcjonowany pod kątem viralowym."""
    start: str = Field(description="Znacznik czasu MM:SS")
    end: str = Field(description="Znacznik czasu MM:SS")
    hook: str = Field(description="Uzasadnienie viralowe")
    score: int = Field(description="Potencjał 1-10") # Usunięto ge/le

class VideoAnalysisReport(BaseModel):
    """Pełny raport z analizy materiału źródłowego."""
    summary: str = Field(description="Główna teza i temat filmu")
    suggested_titles: list[str] = Field(description="3 propozycje tytułów")
    clips: list[VideoClip] = Field(description="Lista fragmentów na Shortsy")

print("LOG: Modele danych zostały uproszczone dla kompatybilności z Gemini Schema.")

LOG: Modele danych zostały uproszczone dla kompatybilności z Gemini Schema.


## 2. Inicjalizacja Agenta Operacyjnego

Używamy prawidłowej składni dla wersji 0.0.18+, gdzie struktura wyjściowa definiowana jest przez parametr `output_type`.

In [3]:
# Inicjalizacja Agenta
agent = Agent(
    model=model,
    output_type=VideoAnalysisReport,  # ✅ Prawidłowa nazwa parametru dla Twojej wersji
    system_prompt=(
        "Jesteś Koordynatorem Operacyjnym w KUŹNI OPERATORÓW. "
        "Twoim zadaniem jest multimodalna analiza wideo i tworzenie briefów montażowych. "
        "Wskaż 3 najlepsze momenty. Wynik musi być zgodny z VideoAnalysisReport."
    )
)

print("LOG: Agent Operacyjny zainicjowany.")

LOG: Agent Operacyjny zainicjowany.


## 3. Zarządzanie Plikami (Native SDK)

Przed analizą plik musi trafić do Google File API i uzyskać status `ACTIVE`.

In [4]:
def upload_and_wait(path: str):
    """Przesyła plik i monitoruje status przetwarzania."""
    print(f"LOG: Przesyłam materiał {path}...")
    file_obj = genai.upload_file(path=path)
    
    while file_obj.state.name == "PROCESSING":
        print(".", end="", flush=True)
        time.sleep(2)
        file_obj = genai.get_file(file_obj.name)
        
    if file_obj.state.name == "FAILED":
        raise RuntimeError("LOG: Błąd krytyczny API.")
        
    print(f"\nLOG: Materiał gotowy do analizy.")
    return file_obj

## 4. Wykonanie Procesu (Analiza i Ekstrakcja)

Uruchamiamy pełny cykl operacyjny. W tej wersji systemu ustrukturyzowane dane wyjściowe z modelu Gemini 2.5 Flash są dostępne przez atrybut `.output` obiektu wynikowego.

In [5]:
import json

# Ścieżka do filmu w katalogu głównym projektu
INPUT_VIDEO = "test_video.mp4"

async def start_mission():
    try:
        # 1. Przesyłanie materiału przez Native SDK
        video_handle = upload_and_wait(INPUT_VIDEO)
        
        # 2. Inicjalizacja modelu
        print("LOG: Agent rozpoczyna analizę z wymuszonym schematem...")
        native_model = genai.GenerativeModel(model_name="gemini-2.5-flash")
        
        # PROMPT operacyjny
        prompt = (
            "Przeanalizuj to wideo. "
            "Zidentyfikuj główny temat oraz wybierz 3 najlepsze fragmenty pod Shortsy. "
            "Zwróć wynik wyłącznie jako JSON zgodny ze strukturą VideoAnalysisReport."
        )
        
        # 3. Wywołanie z obsługą schematu (wymuszenie struktury)
        response = native_model.generate_content(
            [prompt, video_handle],
            generation_config={
                "response_mime_type": "application/json",
                "response_schema": VideoAnalysisReport
            }
        )
        
        # 4. Parsowanie i walidacja po stronie Pythona
        raw_data = json.loads(response.text)
        report = VideoAnalysisReport.model_validate(raw_data)
        
        # 5. Prezentacja wyników
        print("\n" + "="*45)
        print(f"🚀 RAPORT OPERACYJNY: {report.summary.upper()}")
        print("="*45)
        
        print(f"\n💡 SUGEROWANE TYTUŁY:")
        for title in report.suggested_titles:
            print(f"   - {title}")
            
        print(f"\n✂️ PLAN CIĘĆ (SHORTS):")
        for i, clip in enumerate(report.clips, 1):
            print(f"   {i}. [{clip.start} - {clip.end}] (Potencjał: {clip.score}/10)")
            print(f"      HOOK: {clip.hook}\n")
        print("="*45)
            
    except Exception as e:
        print(f"❌ BŁĄD OPERACYJNY: {str(e)}")

# Uruchomienie asynchroniczne
await start_mission()

LOG: Przesyłam materiał test_video.mp4...
.
LOG: Materiał gotowy do analizy.
LOG: Agent rozpoczyna analizę z wymuszonym schematem...

🚀 RAPORT OPERACYJNY: GŁÓWNĄ TEZĄ FILMU JEST PROMOCJA USŁUG FIRMY TAKZEN DEV, OFERUJĄCEJ INTELIGENTNE TWORZENIE STRON INTERNETOWYCH Z WYKORZYSTANIEM TECHNOLOGII TAKICH JAK REACT, NEXT.JS I SZTUCZNA INTELIGENCJA. TEMAT FILMU TO PREZENTACJA ZAAWANSOWANEGO, INNOWACYJNEGO PODEJŚCIA DO WEB DEVELOPMENTU.

💡 SUGEROWANE TYTUŁY:
   - TAKZEN DEV: Inteligentny Web Development z React, Next.js i AI
   - Odkryj Przyszłość Web Dev z TAKZEN DEV i AI!
   - Innowacyjne Rozwiązania Web: React, Next.js, AI od TAKZEN DEV

✂️ PLAN CIĘĆ (SHORTS):
   1. [00:00 - 00:02] (Potencjał: 7/10)
      HOOK: Pierwsze pojawienie się dynamicznego efektu elektrycznego, intrygujące i szybkie wprowadzenie.

   2. [00:02 - 00:05] (Potencjał: 8/10)
      HOOK: Centralna część animacji, gdzie błyskawice w kropli są najbardziej intensywne i symbolizują energię AI.

   3. [00:05 - 00:07] (Potencja